# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
#import gmaps.datasets
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
data = pd.read_csv('city_data.csv')
data.head(3)

,City,City ID,Latitude,Longtitude,Temperature(°C),Humidity(%),Cloudiness(%),Wind Speed (mph)
0,Vaini,4032243,-21.20,-175.20,23.00,78,75,3.60
1,Tiznit Province,2527087,29.58,-9.50,24.99,26,0,1.19
2,Chokurdakh,2126123,70.63,147.92,10.19,49,63,2.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
 gmaps.configure(api_key=g_key)

In [34]:
location = data[["Latitude", "Longtitude"]]
humidity = data["Humidity(%)"]

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(location, weights=humidity)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5

fig.add_layer(heatmap_layer)

#fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Narrow down the DataFrame to find your ideal weather condition. For example:

In [40]:
# A max temperature lower than 80 degrees but higher than 70
ideal_data = data.loc[(data['Temperature(°C)']<80) & (data['Temperature(°C)']>70)]
# Wind speed less than 10 mph
ideal_data = data.loc[data['Wind Speed (mph)'] < 10]
# Zero cloudiness
ideal_data = data.loc[data['Cloudiness(%)'] == 0]

In [46]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
ideal_data = ideal_data.dropna()
ideal_data.head()

,City,City ID,Latitude,Longtitude,Temperature(°C),Humidity(%),Cloudiness(%),Wind Speed (mph)
1,Tiznit Province,2527087,29.58,-9.50,24.99,26,0,1.19
13,Villamontes,3901547,-21.25,-63.50,13.82,63,0,1.81
16,Bilibino,2126682,68.05,166.44,4.33,67,0,2.45
23,Lagoa,3372964,39.05,-27.98,17.50,93,0,7.56
27,Castro,3466704,-24.79,-50.01,5.78,92,0,2.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
hotel_df = pd.DataFrame(ideal_data)
hotel_df['Hotel'] = ""

In [63]:
print(hotel_df['Latitude'])

1      29.58
13    -21.25
16     68.05
23     39.05
27    -24.79
       ...  
491    60.46
496    38.24
515    41.87
519   -22.96
526   -14.83
Name: Latitude, Length: 78, dtype: float64


In [64]:
# Nearby Search required parameters
parameters = {
        "key" : g_key,
        "location" : f"{hotel_df['Latitude']},{hotel_df['Longtitude']}",
        "radius" : 5000,
        "type" : 'hotel'
        }

In [ ]:
# Nearby Search request is an HTTP URL of the following form:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters"

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig